In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm.asyncio import tqdm
import asyncio
import json
import requests

In [2]:
url1 = "https://nextrift.com/sitemap-1.xml"
response1 = requests.get(url1)
soup1 = BeautifulSoup(response1.content, "xml")

url2 = "https://nextrift.com/sitemap-2.xml"
response2 = requests.get(url2)
soup2 = BeautifulSoup(response2.content, "xml")

locs = soup1.find_all('loc')[2:]
locs.extend(soup2.find_all('loc'))
urls = [loc.get_text() for loc in locs]

dataset = [0] * len(urls)

print(len(urls))

3664


In [3]:
async def crawl(x):
    response = requests.get(urls[x])
    soup = BeautifulSoup(response.content, "lxml")
    data = {
        'url': urls[x],
        'title': soup.find('h1', {'class': 'article-title'}).text.strip(),
        'body': soup.find('div', {'class': 'article-content'}).get_text(separator="\n"),
    }
    dataset[x] = data

async def main():
    flist = [crawl(x) for x in range(0, len(urls))]
    for f in tqdm(flist):
        await f

In [4]:
await main()

with open('nextrift.jsonl', 'w') as file:
    for entry in dataset:
        json.dump(entry, file)
        file.write('\n')

100%|██████████████████████████████████████████████████████████████████████████████| 3664/3664 [09:10<00:00,  6.66it/s]
